In [1]:
from fore_utils import *

In [19]:
import torch
import torch.nn as nn
    
class TransformerModel_Point(nn.Module):
    def __init__(self, window_size: int = 24, variables: int = 6, spaces: int = 0, d_model=64, nhead=8, num_layers=8, output_dim: int = 12):
        super(TransformerModel, self).__init__()

        input_dim = window_size * variables * spaces

        self.encoder = nn.Linear(input_dim, d_model)

        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, batch_first=True)

        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)

        self.decoder = nn.Linear(d_model, output_dim)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        # print(x.shape)

        x = self.encoder(x)
        # print(x.shape)

        x = x.unsqueeze(1) 
        
        x = self.transformer_encoder(x)
        # print(x.shape)

        x = self.decoder(x.squeeze(1))
        # print(x.shape)
        
        return x

class TransformerModel_Spatial(nn.Module):
    def __init__(self, window_size: int = 24, variables: int = 6, spaces: int = 0, d_model=64, nhead=8, num_layers=8, output_dim: int = 12):
        super(TransformerModel_Spatial, self).__init__()
        self.spaces = spaces * 2

        input_dim = window_size * variables * self.spaces * self.spaces

        output_dim = window_size * self.spaces * self.spaces

        self.encoder = nn.Linear(input_dim, d_model)

        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, batch_first=True)

        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)

        self.decoder = nn.Linear(d_model, output_dim)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        # print(x.shape)

        x = self.encoder(x)
        # print(x.shape)

        x = x.unsqueeze(1) 
        
        x = self.transformer_encoder(x)
        # print(x.shape)

        x = self.decoder(x.squeeze(1))
        # print(x.shape)
        
        return x.reshape(x.size(0), -1, self.spaces, self.spaces)


In [4]:
window_size = 5
steps = 5
spaces = 2
hidden_size = 32
batch_size = 64
n_heads = 4
num_layers = 8
series_target = False
lightning = True
verbose = True

config = {
    "hidden_layer_sizes": [hidden_size],
    "window_size" : [window_size],
    "step_size" : [steps],
    "spaces" : [spaces],
    "batch_size": [batch_size],
    "n_heads": [n_heads],
    "num_layers": [num_layers],
    "series_target": [series_target],
    "lightning": [lightning],
}

train_set = WeatherData(window_size=window_size, step_size=steps, set='train', spaces=spaces, lightning=lightning, series_target=series_target, verbose=verbose)
train_loader = DataLoader(train_set,
                           batch_size=batch_size, shuffle=True)
    
val_set = WeatherData(window_size=window_size, step_size=steps, set='val', spaces=spaces, lightning=lightning, series_target=series_target, verbose=verbose)
val_loader = DataLoader(val_set,
                           batch_size=batch_size, shuffle=False)


Details for train set:
Data from ['2018', '2019', '2020', '2021'] loaded
Features shape: torch.Size([35064, 4, 4, 6])
Targets shape: torch.Size([35064, 4, 4])
Longitudes: [17.80613  18.056143 18.306158 18.556171]
Latitudes: [-31.137 -31.387 -31.637 -31.887]
Details for val set:
Data from ['2022'] loaded
Features shape: torch.Size([8760, 4, 4, 6])
Targets shape: torch.Size([8760, 4, 4])
Longitudes: [17.80613  18.056143 18.306158 18.556171]
Latitudes: [-31.137 -31.387 -31.637 -31.887]


In [20]:
for batch in train_loader:
    x,b, y = batch
    break

model = TransformerModel_Spatial(window_size=window_size, 
                            variables=6, 
                            spaces=spaces, 
                            d_model=hidden_size, 
                            nhead=n_heads, 
                            num_layers=num_layers, 
                            output_dim=steps)

model.to('cpu')

print('Input shape:', x.shape)
print('Target shape:', y.shape)


y_pred = model(x.float())

print('Prediction shape:', y_pred.shape)

# plt.plot(np.arange(window_size), b[0].detach().numpy(), label='Input')
# plt.plot(np.arange(window_size,window_size+steps), y[0].detach().numpy(),  label='Target')
# plt.plot(np.arange(window_size,window_size+steps), y_pred[0].detach().numpy(), label='Prediction')

# plt.legend()
plt.show()

Input shape: torch.Size([64, 5, 4, 4, 6])
Target shape: torch.Size([64, 5, 4, 4])
Prediction shape: torch.Size([64, 5, 4, 4])


In [23]:
# Hyperparameters
n_epochs = 1
warmup_epochs = 10
initial_lr = 1e-5
early_stopping_patience = 5
checkpoint_path = f'best_model_{window_size}_{hidden_size}_{steps}_{n_heads}_{num_layers}_spatial.pth'

# wandb.init(project="spatial_to_spatial", name=f'model_{window_size}_{hidden_size}_{steps}_spatial_to_spatial', config=config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)
                         
train_model(model, train_loader, val_loader, n_epochs=n_epochs, warmup_epochs=warmup_epochs,
            initial_lr=initial_lr, early_stopping_patience=early_stopping_patience, checkpoint_path=checkpoint_path, device=device)



Epoch 0, Batch 0, Loss: 1.7775


Epoch 0, Batch 100, Loss: 1.6443
Epoch 0, Batch 200, Loss: 1.5745
Epoch 0, Batch 300, Loss: 1.5478
Epoch 0, Batch 400, Loss: 1.7921
Epoch 0, Batch 500, Loss: 1.9393
Epoch 0 Completed, Average Training Loss: 1.6861
Validation Loss after Epoch 0: 1.5224
Learning rate after Epoch 0: 0.000002
New best model saved with validation loss: 1.5224
Best model loaded for evaluation or further use.


In [24]:
test_loader = DataLoader(WeatherData(window_size=window_size, step_size=steps, set='test', spaces=spaces, lightning=lightning, series_target=series_target, verbose=verbose),
                            batch_size=batch_size, shuffle=False)

model.load_state_dict(torch.load(checkpoint_path))
model.eval()

model.to(device)

test_loss = evaluate_model(model, test_loader, device)

print(f"Test loss: {test_loss}")

wandb.log({"test_loss": test_loss})

wandb.finish()

Details for test set:
Data from ['2023'] loaded
Features shape: torch.Size([8760, 4, 4, 6])
Targets shape: torch.Size([8760, 4, 4])
Longitudes: [17.80613  18.056143 18.306158 18.556171]
Latitudes: [-31.137 -31.387 -31.637 -31.887]
Test loss: 1.5135515780344497


epoch,▁
learning_rate,▁
test_loss,▁
train_loss,▁
val_loss,▁
epoch,0
learning_rate,0.0
test_loss,1.51355
train_loss,1.68607
val_loss,1.52245


In [55]:
class WeatherData(Dataset):
    def __init__(self, 
                 window_size: int = 24, 
                 step_size: int = 12, 
                 set:str = 'train', 
                 area: Tuple[int, int] = (-31.667, 18.239), 
                 spaces: int = 0, 
                 intervals: int = 1,
                 lightning: bool = False,
                 series_target: bool = False,
                 verbose: bool = False):
        '''

        Data format:
            0 - humidity
            1 - temperature
            2 - u-wind
            3 - v-wind
            4 - w-wind

        '''
        print('10%', end='\r')

        # Extract correct dataset
        if set == 'train':
            years = ['2018', '2019', '2020', '2021']
        elif set == 'val':
            years = ['2022']
        elif set == 'test':
            years = ['2023']

        if lightning:

            self.data = np.concatenate([np.load(f'forecasting_experiments/datasets/{year}_850_SA.npy') for year in years], axis=1)
            self.data = self.data.transpose(1, 2, 3, 0)

            self.times = np.concatenate([np.load(f'forecasting_experiments/datasets/{year}_850_SA_times.npy') for year in years])

            self.lon = np.load('forecasting_experiments/datasets/SA_lon.npy')
            self.lat = np.load('forecasting_experiments/datasets/SA_lat.npy')

        else:
            self.data = np.concatenate([np.load(f'datasets/{year}_850_SA.npy') for year in years], axis=1)
            self.data = self.data.transpose(1, 2, 3, 0)

            self.times = np.concatenate([np.load(f'datasets/{year}_850_SA_times.npy') for year in years])

            self.lon = np.load('datasets/SA_lon.npy')
            self.lat = np.load('datasets/SA_lat.npy')

        print('40%', end='\r')

        # Get lat and long

        self.spaces = spaces

        self.get_area(area)

        print('50%', end='\r')

        self.series_target = series_target

        # Normalize data and sort into variables

        if spaces != 0:
            q = self.data[:, :, :, 0]
            t = self.data[:, :, :, 1]
            u = self.data[:, :, :, 2]
            v = self.data[:, :, :, 3]
            w = self.data[:, :, :, 4]
        else:
            q = self.data[:,0]
            t = self.data[:,1]
            u = self.data[:,2]
            v = self.data[:,3]
            w = self.data[:,4]
            self.series_target = False

        q, t, u, v, w = self.normalize(q, t, u, v, w)

        print('70%', end='\r')

        # Calculate wind speed and direction
        

        self.calculate_wind(u, v)

        print('90%', end='\r')

        # Serup the dataloader
        if self.series_target:
            self.features = torch.tensor(np.stack([q, t, u, v, w], axis=-1), dtype=torch.float32)
        else:
            self.features = torch.tensor(np.stack([q, t, u, v, w, self.wspd], axis=-1), dtype=torch.float32)

        self.targets = torch.tensor(self.wspd, dtype=torch.float32)
        self.window_size = window_size
        self.step_size = step_size

        print('100%', end='\r')

        self.intervals = intervals

        if verbose:
            print(f'Details for {set} set:')

            print(f'Data from {years} loaded')
            
            print(f'Features shape: {self.features.shape}')
            print(f'Targets shape: {self.targets.shape}')

            print(f'Longitudes: {self.lon}')
            print(f'Latitudes: {self.lat}')

    def __len__(self):
        return self.data.shape[0] - self.window_size - self.step_size + 1

    def __getitem__(self, idx):
            return self.features[idx : idx + self.window_size], self.targets[idx : idx + self.window_size],self.targets[idx + self.window_size : idx + self.window_size + self.step_size]
    
    def normalize(self, q, t, u, v, w, method = 'std'): 
        if method == 'std':
            q = (q - q.mean()) / q.std()
            t = (t - t.mean()) / t.std()
            u = (u - u.mean()) / u.std()
            v = (v - v.mean()) / v.std()
            w = (w - w.mean()) / w.std()

        return q, t, u, v, w        
    
    def calculate_wind(self, u, v):
        if self.series_target:
            u = u[:, u.shape[1]//2, u.shape[2]//2]
            v = v[:, v.shape[1]//2, v.shape[2]//2]

        self.wspd = np.sqrt(u**2 + v**2)
        self.wdir = np.arctan2(u, v)

    def get_area(self, area: Tuple[int, int]):
        lon = np.argmin(np.abs(self.lon - area[1]))

        lat = np.argmin(np.abs(self.lat - area[0]))

        if self.spaces != 0:

            self.lon = self.lon[lon - self.spaces:lon + self.spaces]
            self.lat = self.lat[lat - self.spaces: lat + self.spaces]

            self.data = self.data[:, lat - self.spaces: lat + self.spaces, lon - self.spaces:lon + self.spaces, :]
        else:

            self.lon = self.lon[lon]
            self.lat = self.lat[lat]

            self.data = self.data[:, lat, lon, :]

    def plot_area(self):
        if self.spaces != 0:
            fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()})

            ax.coastlines()

            ax.set_extent([self.lon.min(), self.lon.max(), self.lat.min(), self.lat.max()])

            lon, lat = self.lon, self.lat
            contour = ax.contourf(lon, lat, self.targets[0].detach().numpy(), transform=ccrs.PlateCarree())

            fig.colorbar(contour, ax=ax, orientation='vertical', label='Wind Speed (m/s)')

            plt.show()
        else:
            print('Cannot plot area with only one point')

    def plot_animation(self, seed: int = 0, frame_rate: int = 16, levels: int = 10) -> HTML:
        """
        Plots features and targets from the windowed arrays for visualization.

        Args:
            seed (int): Seed for reproducibility in selecting samples. Default is 0.
            frame_rate (int): The frame rate for the animation. Default is 16.
            levels (int): Number of contour levels for the plot. Default is 10.

        Returns:
            HTML: An HTML object representing the animation.
        """
        if self.spaces != 0:
            bounds = [self.lon.min(), self.lon.max(), self.lat.min(), self.lat.max()]

            features = self.features[seed:seed + self.window_size * self.intervals:self.intervals]
            targets = self.targets[seed + self.window_size * self.intervals:seed + (self.window_size + self.step_size) * self.intervals: self.intervals]
            
            time_features = self.times[seed:seed + self.window_size * self.intervals:self.intervals]
            time_targets = self.times[seed + self.window_size * self.intervals:seed + (self.window_size + self.step_size) * self.intervals: self.intervals]

            time_features = pd.to_datetime(time_features)
            time_targets = pd.to_datetime(time_targets)

            fig, axs = plt.subplots(1, 2, figsize=(21, 7), subplot_kw={'projection': ccrs.PlateCarree()})

            vmin = min(features.min().item(), targets.min().item())
            vmax = max(features.max().item(), targets.max().item())

            fig.subplots_adjust(left=0.05, right=0.95, bottom=0.1, top=0.9, wspace=0.2)

            for ax in axs:
                ax.set_extent(bounds, crs=ccrs.PlateCarree())
                ax.coastlines()

            feat = axs[0].contourf(self.lon, self.lat, features[0, :, :, 2], levels=levels, vmin=vmin, vmax = vmax, transform=ccrs.PlateCarree())
            tar = axs[1].contourf(self.lon, self.lat, targets[0], levels=levels, vmin=vmin, vmax = vmax, transform=ccrs.PlateCarree())
            axs[1].set_title('Target')

            fig.colorbar(feat, ax=axs[0], orientation='vertical', label='Wind Speed (m/s)')
            fig.colorbar(tar, ax=axs[1], orientation='vertical', label='Wind Speed (m/s)')

            def animate(i):
                axs[0].clear()
                axs[0].coastlines()

                axs[0].contourf(self.lon, self.lat, features[i, :, :, 2], levels=levels, vmin=vmin, vmax = vmax)

                axs[0].set_title(f'Window {i} - {time_features[i].strftime("%Y-%m-%d %H:%M:%S")}')
                if self.step_size > 1:
                    axs[1].contourf(self.lon, self.lat, targets[i % self.step_size], levels=levels, vmin=vmin, vmax = vmax)
                    axs[1].set_title(f'Target - {time_targets[i % self.step_size].strftime("%Y-%m-%d %H:%M:%S")}')
                # return pcm

                
            frames = features.shape[0]

            interval = 1000 / frame_rate

            ani = FuncAnimation(fig, animate, frames=frames, interval=interval)

            plt.close(fig)

            return HTML(ani.to_jshtml())
        else:
            print('Cannot plot area with only one point')

    def plot_point(self, seed: int = 0):
        if self.spaces == 0:
            plt.figure(figsize=(10, 5))

            plt.plot(self.times[seed:seed + self.window_size], self.features[seed:seed + self.window_size, 0], label='Humidity')
            plt.plot(self.times[seed:seed + self.window_size], self.features[seed:seed + self.window_size, 1], label='Temperature')
            plt.plot(self.times[seed:seed + self.window_size], self.features[seed:seed + self.window_size, 2], label='U-Wind')
            plt.plot(self.times[seed:seed + self.window_size], self.features[seed:seed + self.window_size, 3], label='V-Wind')
            plt.plot(self.times[seed:seed + self.window_size], self.features[seed:seed + self.window_size, 4], label='W-Wind')
            plt.plot(self.times[seed:seed + self.window_size], self.features[seed:seed + self.window_size, 5], label='Wind Speed', linestyle='--')

            plt.plot(self.times[seed + self.window_size:seed + self.window_size + self.step_size], self.targets[seed + self.window_size:seed + self.window_size + self.step_size], label='Wind Speed', linestyle='--')
            
            plt.xticks(rotation=45)
            plt.legend()
            plt.show()
        else:
            print('Cannot plot point with multiple points')           

    def plot_prediction(self, model, seed: int = 0, frame_rate: int = 16, levels: int = 10) -> HTML:
        if self.spaces != 0:
            bounds = [self.lon.min(), self.lon.max(), self.lat.min(), self.lat.max()]

            features = self.features[seed:seed + self.window_size * self.intervals:self.intervals]
            targets = self.targets[seed + self.window_size * self.intervals:seed + (self.window_size + self.step_size) * self.intervals: self.intervals].detach().numpy()   

            predictions = model(features.unsqueeze(0)).detach().numpy().squeeze()
            
            print(predictions.shape)

            time_features = self.times[seed:seed + self.window_size * self.intervals:self.intervals]
            time_targets = self.times[seed + self.window_size * self.intervals:seed + (self.window_size + self.step_size) * self.intervals: self.intervals]

            time_targets = pd.to_datetime(time_targets)

            
            fig, axs = plt.subplots(2, 3, figsize=(21, 7), subplot_kw={'projection': ccrs.PlateCarree()})

            vmin = min(predictions.min().item(), targets.min().item())
            vmax = max(predictions.max().item(), targets.max().item())

            fig.subplots_adjust(left=0.05, right=0.95, bottom=0.1, top=0.9, wspace=0.2)

            for ax in axs.flatten()[:-1]:
                ax.set_extent(bounds, crs=ccrs.PlateCarree())
                ax.coastlines()

            ax_last = fig.add_subplot(2, 3, 6)

            print('Predictions:', predictions.shape)
            print('Targets:', targets.shape)

            pred = axs[0, 0].contourf(self.lon, self.lat, predictions[0], levels=levels, vmin=vmin, vmax = vmax, transform=ccrs.PlateCarree())
            tar = axs[0, 1].contourf(self.lon, self.lat, targets[0], levels=levels, vmin=vmin, vmax = vmax, transform=ccrs.PlateCarree())

            error = (predictions[0] - targets[0].squeeze()) 

            err = axs[0, 2].contourf(self.lon, self.lat, error.squeeze(), levels=levels, transform=ccrs.PlateCarree(), cmap='coolwarm')

            perc_error = error / targets[0,0].squeeze() * 100
            perc_error = np.clip(perc_error, -100, 100)
            rmse = np.sqrt(error**2)

            perr = axs[1, 0].contourf(self.lon, self.lat, perc_error, levels=levels, transform=ccrs.PlateCarree(), cmap='coolwarm')
            rms = axs[1, 1].contourf(self.lon, self.lat, rmse, levels=levels, transform=ccrs.PlateCarree(), cmap='coolwarm')
            ax_last.scatter(targets[0].flatten(), predictions[0].flatten(), c=error, cmap='coolwarm')

            fig.colorbar(pred, ax=axs[0, 0], orientation='vertical', label='Wind Speed (m/s)')
            fig.colorbar(tar, ax=axs[0, 1], orientation='vertical', label='Wind Speed (m/s)')
            fig.colorbar(err, ax=axs[0, 2], orientation='vertical', label='Percentage Error (%)')
            fig.colorbar(perr, ax=axs[1, 0], orientation='vertical', label='Percentage Error (%)')
            fig.colorbar(rms, ax=axs[1, 1], orientation='vertical', label='Root Mean Squared Error (m/s)')

            ax_last.set_xlabel("Observed Wind Speed (m/s)")
            ax_last.set_ylabel("Forecasted Wind Speed (m/s)")

            def animate(i):
                for ax in axs.flatten()[:-1]:
                    ax.clear()
                    ax.coastlines()
                
                ax_last.clear()
                ax_last.set_xlabel("Observed Wind Speed (m/s)")
                ax_last.set_ylabel("Forecasted Wind Speed (m/s)")

                axs[0, 0].contourf(self.lon, self.lat, predictions[i], levels=levels, vmin=vmin, vmax = vmax)
                axs[0, 1].contourf(self.lon, self.lat, targets[i], levels=levels, vmin=vmin, vmax = vmax)
                
                error =  (predictions[i] - targets[i].squeeze())
                axs[0, 2].contourf(self.lon, self.lat, error, levels=levels, transform=ccrs.PlateCarree(), cmap='coolwarm')
                
                perc_error = error / targets[i % self.step_size].squeeze() * 100
                perc_error = np.clip(perc_error, -100, 100)
                rmse = np.sqrt(error**2)

                axs[1, 0].contourf(self.lon, self.lat, perc_error, levels=levels, transform=ccrs.PlateCarree(), cmap='coolwarm')
                axs[1, 1].contourf(self.lon, self.lat, rmse, levels=levels, transform=ccrs.PlateCarree(), cmap='coolwarm')
                ax_last.scatter(targets[i].flatten(), predictions[i].flatten(), c=error, cmap='coolwarm')

                axs[0, 0].set_title(f'Prediction {i} - {time_targets[i].strftime("%Y-%m-%d %H:%M:%S")}')  
                axs[0, 1].set_title(f'Target - {time_targets[i].strftime("%Y-%m-%d %H:%M:%S")}')
                axs[0, 2].set_title(f'Error - {time_targets[i].strftime("%Y-%m-%d %H:%M:%S")}')
                axs[1, 0].set_title(f'Percentage Error - {time_targets[i].strftime("%Y-%m-%d %H:%M:%S")}')
                axs[1, 1].set_title(f'Root Mean Squared Error - {time_targets[i].strftime("%Y-%m-%d %H:%M:%S")}')
                ax_last.set_title(f'Error Scatter Plot - {time_targets[i].strftime("%Y-%m-%d %H:%M:%S")}')

            frames = predictions.shape[0]

            interval = 1000 / frame_rate

            ani = FuncAnimation(fig, animate, frames=frames, interval=interval)

            plt.close(fig)

            return HTML(ani.to_jshtml())

        else:
            print('Cannot plot area with only one point')


In [67]:
spaces = 2

train_set = WeatherData(window_size=window_size, step_size=steps, set='train', spaces=spaces, lightning=lightning, series_target=series_target, verbose=verbose)
train_loader = DataLoader(train_set,
                           batch_size=batch_size, shuffle=True)


Details for train set:
Data from ['2018', '2019', '2020', '2021'] loaded
Features shape: torch.Size([35064, 4, 4, 6])
Targets shape: torch.Size([35064, 4, 4])
Longitudes: [17.80613  18.056143 18.306158 18.556171]
Latitudes: [-31.137 -31.387 -31.637 -31.887]


In [68]:

for batch in train_loader:
    x,b, y = batch
    break

model = TransformerModel_Spatial(window_size=window_size, 
                            variables=6, 
                            spaces=spaces, 
                            d_model=hidden_size, 
                            nhead=n_heads, 
                            num_layers=num_layers, 
                            output_dim=steps)

model.to('cpu')

y_pred = model(x.float())

print('Input shape:', x.shape)
print('Target shape:', y.shape)
print('Prediction shape:', y_pred.shape)

train_set.plot_prediction(model, seed=0, frame_rate=8, levels=spaces)

Input shape: torch.Size([64, 5, 4, 4, 6])
Target shape: torch.Size([64, 5, 4, 4])
Prediction shape: torch.Size([64, 5, 4, 4])
(5, 4, 4)
Predictions: (5, 4, 4)
Targets: (5, 4, 4)
